# Ensemble Learning

## Initial Imports

In [124]:
import warnings
warnings.filterwarnings('ignore')

In [125]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [126]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [127]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [128]:
# Encode categorical data

yn_binary = {
    "N":0,
    "n":0,
    "Y":1,
    "y":1
}
df.drop(columns=["issue_d", "initial_list_status", "next_pymnt_d"], inplace=True)
df["pymnt_plan"] = df["pymnt_plan"].apply(lambda x: yn_binary[x])
df["hardship_flag"] = df["hardship_flag"].apply(lambda x: yn_binary[x])
df["debt_settlement_flag"] = df["debt_settlement_flag"].apply(lambda x: yn_binary[x])
df = pd.get_dummies(df, columns=['home_ownership', "verification_status", "application_type"])
new_cols = [col for col in df.columns if col != 'loan_status'] + ['loan_status']
df = df[new_cols]
df

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,loan_status
0,10500.0,0.1719,375.35,66000.0,0,27.24,0.0,0.0,8.0,0.0,...,0,0,0,1,0,1,0,1,0,low_risk
1,25000.0,0.2000,929.09,105000.0,0,20.23,0.0,0.0,17.0,1.0,...,0,1,0,0,0,0,1,1,0,low_risk
2,20000.0,0.2000,529.88,56000.0,0,24.26,0.0,0.0,8.0,0.0,...,0,1,0,0,0,0,1,1,0,low_risk
3,10000.0,0.1640,353.55,92000.0,0,31.44,0.0,1.0,10.0,1.0,...,0,0,0,1,0,0,1,1,0,low_risk
4,22000.0,0.1474,520.39,52000.0,0,18.76,0.0,1.0,14.0,0.0,...,0,1,0,0,1,0,0,1,0,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,0,9.60,0.0,0.0,9.0,0.0,...,0,0,0,1,0,1,0,1,0,low_risk
68813,12000.0,0.2727,368.37,63000.0,0,29.07,0.0,0.0,8.0,0.0,...,0,0,0,1,1,0,0,1,0,low_risk
68814,5000.0,0.1992,185.62,52000.0,0,14.86,0.0,0.0,5.0,1.0,...,0,1,0,0,0,1,0,1,0,low_risk
68815,40000.0,0.0646,1225.24,520000.0,0,9.96,0.0,1.0,21.0,0.0,...,0,1,0,0,0,0,1,1,0,low_risk


## Split the Data into Training and Testing

In [129]:
# Create our features
X = df.drop(columns=["loan_status"])

# Create our target
y = df["loan_status"]

In [130]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.0,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.0,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,0.0,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.0,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.0,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,0.0,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,0.0,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,0.0,999.000000,18.000000,5.000000,72.000000,4.000000,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [131]:
# Check the balance of our target values
y.value_counts()


low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [132]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [133]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [134]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [135]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [153]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [154]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7162591229890061

In [155]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,60,41
Actual 1,2763,14341


In [139]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.59      0.84      0.04      0.71      0.49       101
   low_risk       1.00      0.84      0.59      0.91      0.71      0.51     17104

avg / total       0.99      0.84      0.60      0.91      0.71      0.51     17205



In [145]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.06661063199651111, 'total_rec_prncp'),
 (0.06447914987224355, 'total_rec_int'),
 (0.060514529390235446, 'last_pymnt_amnt'),
 (0.055786666336777126, 'total_pymnt'),
 (0.05499403538538883, 'total_pymnt_inv'),
 (0.03278388496286508, 'int_rate'),
 (0.020070463868758413, 'dti'),
 (0.019051745686531698, 'mths_since_recent_inq'),
 (0.018101122244224218, 'installment'),
 (0.017263161088940982, 'max_bal_bc'),
 (0.016756867104044063, 'annual_inc'),
 (0.01632514492330037, 'out_prncp'),
 (0.016196733058178883, 'mo_sin_old_rev_tl_op'),
 (0.01602232548695511, 'tot_hi_cred_lim'),
 (0.01588272916700662, 'il_util'),
 (0.015770475916786358, 'mths_since_rcnt_il'),
 (0.01574636928567132, 'revol_bal'),
 (0.015346168025129628, 'out_prncp_inv'),
 (0.01529756139089808, 'mo_sin_old_il_acct'),
 (0.014993791415145792, 'total_bc_limit'),
 (0.014671569399516488, 'tot_cur_bal'),
 (0.01455732776632548, 'bc_util'),
 (0.014457073362670502, 'bc_open_to_buy'),
 (0.014297459800644922, 'avg_cur_bal'),
 (0.014172439434

### Easy Ensemble Classifier

In [146]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=10, random_state=1)
ee_model = ee_model.fit(X_train_scaled, y_train)
y_pred = ee_model.predict(X_test_scaled)

In [147]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7064886680436051

In [151]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,68,33
Actual 1,4452,12652


In [152]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.67      0.74      0.03      0.71      0.49       101
   low_risk       1.00      0.74      0.67      0.85      0.71      0.50     17104

avg / total       0.99      0.74      0.67      0.84      0.71      0.50     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Balanced Random Forest Classifier

2. Which model had the best recall score?

    Balanced Random Forest had the best overall recall score, however Easy Ensemble Classifier had better recall when it came to correctly labeling high risk.

3. Which model had the best geometric mean score?

    Both ensemble methods had equivalent geometric mean scores on this dataset. 

4. What are the top three features?
    
    The top three features in order of their weights:
    1. total_rec_prncp (0.06661063199651111)
    2. total_rec_int (0.06447914987224355)
    3. last_pymnt_amnt (0.060514529390235446)